In [ ]:
'''
For prior assumpsion, we take the medically known values and some rough assumptions on our behalf-
For y=0
    Age - 30 (Younger people usually have higher tendency)
    Rest Blood Pressure - 120 (According to my understanding of what I obtained from the internet)
    Cholestrol Level - 162.5 (125 to 200 - we bluntly take the average, therefore, = 162.5)
    
For y=1
    Age - 60 
    Rest Blood Pressure - 150 
    Cholestrol Level - 200 

Variance we randomly take as 1 across all features and covariance as 0 (We will vary it and see the results)
'''

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re

In [2]:
def import_dataset(split_percent = 70):
    dataset = pd.read_csv('E:\ELL_project\problem1\health_data.csv')
    dataset = dataset.sample(frac = 1)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1:].values
    datasize = X.shape[0]

    split_point = split_percent//10

    X_train = X[:(datasize*split_point)//10,:]
    y_train = y[:(datasize*split_point)//10,:]
    X_test = X[(datasize*split_point)//10:,:]
    y_test = y[(datasize*split_point)//10:,:]

    return(X_train,X_test,y_train,y_test)


In [ ]:
def feature_scaling(X_train,X_test):
    X_mean = np.sum(X_train,axis=0)
    X_var = np.sqrt(np.sum(np.square(X_train - X_mean), axis=0))

    X_train_feat_scaled = (X_train - X_mean ) / X_var
    X_test_feat_scaled = (X_test - X_mean) / X_var

    return (X_mean,X_var, X_train_feat_scaled,X_test_feat_scaled)

In [1]:
def sep_01(X_train,y_train):
    x_0_train = X_train[y_train[:,0]==0]
    x_1_train = X_train[y_train[:,0]==1]
    return(x_0_train,x_1_train)

In [2]:
def mean_var(x):
    siz = x.shape[0]
    mean = np.sum(x,axis=0)/siz
    dif = X-mean
    covariance_matrix = np.dot(np.transpose(dif),dif) / siz

    return(mean,covariance_matrix)

In [ ]:
def gen_meu_var(m1,m2,m3,v1,v2,v3):
    prior_mean = np.array([m1,m2,m3])
    prior_variance = np.array([v1,v2,v3])
    return(prior_mean,prior_variance)

In [ ]:
def ret_n(meu0,meu,cov0,cov,data_size):
    # Now, we will consider L as the left most value, M as the middle one and R as the right one in the formula for meu        # and cov matrix
    l = cov0
    m = np.inverse(cov0 + (cov/data_size))
    r = cov/data_size

    meu_n = np.dot(np.dot(l,m)meu) + np.dot(np.dot(r,m),meu0)
    cov_n = np.dot(np.dot(l,m),r)

    return (meu_n,cov_n)

In [1]:
def mean_var_data(X_train,y_train,prior_vals):
    x_0,x_1 = sep_01(X_train,y_train)
    meu_0,covar_0 = mean_var(x_0)
    meu_1,covar_1 = mean_var(x_1)

    prior_mean_var = gen_meu_var(prior_vals)
    
    prior_mean_0 = prior_mean_var[0]
    prior_covar_0 = prior_mean_var[1]
    prior_mean_1 = prior_mean_var[2]
    prior_covar_1 = prior_mean_var[3]

    size_0 = x_0.shape[0]
    meu_n_0,cov_n_0 = ret_n(prior_mean_0,meu_0,prior_covar_0,covar_0,size_0)
    size_1 = x_1.shape[0]
    meu_n_1,cov_n_1 = ret_n(prior_mean_1,meu_1,prior_covar_1,covar_1,size_1)

    return(meu_n_0,cov_n_0, meu_n_1,cov_n_1)


In [ ]:
def eval_on_test()